# Demonstration of multipath fading

This demo is supposed to visualize the effect of [multipath](https://en.wikipedia.org/wiki/Multipath_propagation) [fading](https://en.wikipedia.org/wiki/Fading) in a wireless transmission scenario.

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on [MIMO transmission](https://en.wikipedia.org/wiki/MIMO) hold at the [university of applied siences, Berlin](https://www.htw-berlin.de/).

## Import libraries and set defaults

In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.style.use('noelle.mplstyle')

In [2]:
def rotate(x, y, phi):
    xr = x*np.cos(phi) + y*-np.sin(phi)
    yr = x*np.sin(phi) + y*np.cos(phi)
    
    return xr, yr

In [3]:
def plotWaves(x, y, axLim, fig, label=None):

    fig = plt.figure(fig)    
    ax = fig.add_axes((1,1,1,1), label=label)
    ax.plot(x, y)
    ax.axis((-2, axLim+2, (-axLim+4)/2, (axLim+4)/2))
    ax.grid(True)
    ax.set_aspect('equal','box')

## Frequency selective fading in a simplified two-path scenario

For the sake of simplicity, only two propagation paths are considered here. That means that the signal from the transmit antenna propagates via a direct path ([line of sight, LOS](https://en.wikipedia.org/wiki/Line-of-sight_propagation)) on the one hand and via a reflected path ([non line of sight, NLOS](https://en.wikipedia.org/wiki/Non-line-of-sight_propagation)) on the other and these two signals combine at the receive antenna. This scenario is depicted below: The transmitter and receiver are located at point A and B, respectively and the reflection of the NLOS signale takes place at point C.

![Dreieck](https://upload.wikimedia.org/wikipedia/commons/d/dd/Dreieck.svg)

In this demonstration, the propagation delay ($\Delta\tau$) between the LOS and the NLOS path, their amplitude ratio ($\Delta a$) and the frequency ($f$) of transmitted wave can be varied. 

It can be observed that the two waves do interfere either constructively or destructively at the receive antenna, depending on their particular frequency. This results in either a large or a small resulting signal amplitude at the reciever. This effect that the propagation channel treats various signal frequencies differently (amplifies some and attenuates others) is called [frequency-selective fading](https://en.wikipedia.org/wiki/Fading#Selective_fading) and is (amongst other effects) responsible for signal distortions in a wireless channel.

By changing the propagation delay between the two paths, the frequencies at which the resulting signal is attenuated or amplified changes. Meaning that a different delay completely changes the behaviour of the wireless transmission channel. In this simplified example of only two interacting waves, the frequency dependent pattern of the received signal amplitude is periodic and cosine shaped. The frequency location of the minima and maxima of the received signal amplitude depend on the actual delay $\Delta\tau$. Further, the heights of the minima and maxima depend on the ratio of hte amplitudes of the interacting waves $\Delta a$.

In [4]:
def plot_multipath_fading(d_tau, f, d_amp):

    d_x = 1e-3 # step size in m
    v = 1 # velocity of the waves in m/s
    amp_los = 0.5 # amplitude of los wave
    amp_nlos = amp_los * d_amp # amplitude of nlos wave
    ratio_b_a = 0.3 # length ratio between path a and b
    c = 10 # length of path c (LOS)

    b = ratio_b_a*(c+v*d_tau)
    a = (c+v*d_tau) - b
    B_x = c
    B_y = 0

    df = 1/((a+b)/v)  
    frequencies = np.arange(df/4,2,df/10)

    alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
    beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

    interference = np.ones_like(frequencies)
    # index of selected frequency
    f_idx = np.argmin(np.abs(frequencies-f))

    for idx, frequency in enumerate(frequencies):

        k = 2*np.pi/v*frequency # 2*pi/lambda   
        tmp_b_x, tmp_b_y = rotate(np.arange(0,b,d_x), amp_los*np.sin(k*np.arange(0,b,d_x)), alpha)
    
        C_x, C_y = rotate(b, 0, alpha);
        tmp_a_x, tmp_a_y = rotate(np.arange(0,a,d_x), amp_nlos*np.sin(k*(np.arange(0,a,d_x)+b)), -beta)
        tmp_a_x = tmp_a_x + C_x
        tmp_a_y = tmp_a_y + C_y

        tmp_c_x = np.arange(0,c,d_x)
        tmp_c_y = amp_los*np.sin(k*np.arange(0,c,d_x))
                
        interference[idx] = np.abs(amp_los*np.exp(1j*k*((a+b)-d_x)) + amp_nlos*np.exp(1j*k*(c-d_x)))
        
        if idx == f_idx:
            # save wave at selected frequency
            w_a_x, w_a_y, w_b_x, w_b_y, w_c_x, w_c_y = tmp_a_x, tmp_a_y, tmp_b_x, tmp_b_y, tmp_c_x, tmp_c_y
            los = amp_los*np.exp(1j*k*((a+b)-d_x))
            nlos = amp_nlos*np.exp(1j*k*(c-d_x))

    reflector_x, reflector_y = rotate(np.arange(-2,2,0.1), 0, (alpha-beta)/2)
    reflector_x = reflector_x + C_x
    reflector_y = reflector_y + C_y
    
    # plotting
    n_row = 1
    n_col = 3
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)
    fig.clf()
    ax = fig.add_subplot(n_row,n_col,1)
    ax.plot(w_a_x,  w_a_y, 'C0')
    ax.plot(w_b_x,  w_b_y, 'C0')
    ax.plot(w_c_x,  w_c_y, 'C1')
    ax.text(-1.5,-0.5,'Tx', size=16)
    ax.text(B_x,-0.5,'Rx', size=16)    
    ax.axis([-2, c+2, (-c+4)/2, (c+4)/2])    
    plt.plot(reflector_x,  reflector_y, 'k')
    ax.axis('off')
    
    axins = inset_axes(ax, width=1, height=1)
    axins.plot(w_a_x,  w_a_y, 'C0')
    axins.plot(w_c_x,  w_c_y, 'C1')
    axins.plot(np.ones(20)*B_x, np.arange(-1,1,0.1), 'C1')
    tmp_x, tmp_y = rotate(np.zeros(20), np.arange(-1,1,0.1), -beta)
    axins.plot(tmp_x+B_x, tmp_y+B_y, 'C0')
    axins.plot(w_a_x[-1], w_a_y[-1], 'C0o', ms=10)
    axins.plot(w_c_x[-1], w_c_y[-1], 'C1o', ms=10)
    axins.axis([B_x-0.5, B_x+0.5, B_y-0.5, B_y+0.5])
    axins.set_xticklabels([])
    axins.set_yticklabels([])
    axins.set_xticks([])
    axins.set_yticks([])
    
    ax = fig.add_subplot(n_row,n_col,2)
    a1 = ax.arrow(0,0,los.real, los.imag, color='C1', width=0.015, length_includes_head=True, label='LOS signal')
    a2 = ax.arrow(los.real, los.imag,nlos.real, nlos.imag, color='C0', width=0.015, length_includes_head=True, label='NLOS signal')
    a3 = ax.arrow(0, 0,nlos.real+los.real, nlos.imag+los.imag, color='C3', width=0.025, length_includes_head=True, label='combined signal')
    ax.axis([-1, 1, -1, 1])
    ax.set_xlabel('real part')
    ax.set_ylabel('imaginary part')
    ax.legend([a1, a2, a3], ['LOS signal', 'NLOS signal', 'combined signal'], loc=3)

    ax = fig.add_subplot(n_row,n_col,3)
    ax.plot(frequencies, abs(interference),ls='-', color=(0.8,0.8,0.8))
    ax.plot(f, abs(interference[f_idx]),'C3o')
    ax.axis([0, 2, 0, 1])
    ax.set_xlabel('frequency / Hz')
    ax.set_ylabel('amplitude at Rx')   
    
# sliders 
style = {'description_width': '30%'}
w_d_tau = widgets.FloatSlider(min=0, max=8, step=1, value=2.0, continuous_update=False, description=r'$\Delta \tau:$', style=style)
w_d_amp = widgets.FloatSlider(min=0, max=1.0, step=0.1, value=1.0, continuous_update=False, description=r'$\Delta a:$', style=style)
#w_c = widgets.FloatSlider(min=0, max=20, step=1, value=10.0, continuous_update=False, description=r'$c:$', style=style)
#w_ratio_b_a = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.33, continuous_update=False, description='ratio b/a:', style=style)
w_f = widgets.FloatSlider(min=0, max=2, step=0.01, value=0.1, continuous_update=False, description=r'$f:$', style=style)
   
ui = widgets.HBox([w_d_tau, w_f, w_d_amp])
out = widgets.interactive_output(plot_multipath_fading, {'d_tau':w_d_tau, 'f':w_f, 'd_amp':w_d_amp})

out.layout.height = '350px'

display(ui, out)

Output(layout=Layout(height='350px'))

## Frequency selective fading in a "real" multipath scenario

In the more realistic case of multiple NLOS paths, each having different amplidues and delays, the pattern of the frequency selectiveness of the channel is more complex (compare lecture slides).

## Frequency selective fading in combination with the Doppler effect

The [Doppler effect](https://en.wikipedia.org/wiki/Doppler_effect) causes a frequency shift of the received signal, while the amount of this shift is dependent on the relative velocity and direction of the movement between transmitter and receiver. In a dynamic scenario of moving receive devices (smartphones and handhelds) the frequency of the received LOS and NLOS signals are therefore constantly changing, even when considering a fixed, constant transmit frequency. In combination with the effect of frequency selective fading visualized above, this results in a time varying amplitude of the received signal, commonly refferred to as [fast fading](https://en.wikipedia.org/wiki/Fading#Slow_versus_fast_fading).

Two examples of this time varying amplitude fluctuations are shown below:

<img src="https://upload.wikimedia.org/wikipedia/commons/e/e0/Rayleigh_fading_doppler_10Hz.svg" width="30%"> <img src="https://upload.wikimedia.org/wikipedia/commons/5/5c/Rayleigh_fading_doppler_100Hz.svg" width="30%">
